In [43]:
# -*- coding: utf-8 -*-
"""Estimate costs and benefits, either under fixed parameters or under a sensitivity analysis,
varying the cost components.
"""
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

Argentina Cost Preprocessing [incomplete]

QUESTION: We only have data for roads, what about rails? We also do not have min and max data, should it be multiplied by a factor? I also wanted to check that we are using the climate uplift cost. 

In [64]:
# Define paths
incoming_data_path = "/Volumes/IT/git_projects/traction/casestudies/argentina/incoming_data"
road_path = "/Volumes/IT/git_projects/traction/casestudies/argentina/data/networks/rail.gpkg"

In [62]:
adapt = pd.read_excel(os.path.join(incoming_data_path,'costs','ROCKS - Database - ARNG (Version 2.3) Feb2018.xls'),
            sheet_name = 'Resultados Consolidados',
            skiprows=6,
            nrows=9,
            usecols = [2,4,5]).fillna('No value')
adapt.columns = ['option','cost_perkm','climate_uplift_perkm']
adapt = adapt[~adapt.option.isin(['Subtotal','No value'])]

cost_rehab = adapt.loc[adapt['option']=='Reconstruction','cost_perkm'].values[0] + \
                adapt.loc[adapt['option']=='Reconstruction','climate_uplift_perkm'].values[0]

In [69]:
# Add road costs
road_edges = gpd.read_file(road_path, layer = "edges")

# Set cost_unit
road_edges["cost_unit"] = "USD/km"

# Set cost_min
road_edges["cost_min"] = cost_rehab * 1000

# Set cost_max
road_edges["cost_max"] = cost_rehab * 1000

### Export to file
road_edges.to_file(road_path, layer='edges', driver='GPKG')

Vietnam Cost Preprocessing [incomplete]

QUESTION: Cost assigning code uses asset_type column but this column is empty in the network. Am I looking in the wrong place or was this generated in a different way?

In [ ]:
def damage_costs_per_area_vietnam(x, rehab_costs,length_factor,national=False):
    """Estimate the total cost and benefits for a road segment. This function is used within a
    pandas apply

    Parameters
    ----------
    x
        a row from the road segment dataframe that we are considering
    rehab_costs
        rehabilitation costs after a disaster

    Returns
    -------
    uncer_output : list
        outcomes for the initial adaptation costs of this road segment
    tot_uncer_output : list
        outcomes for the total adaptation costs of this road segment
    rel_share : list
        relative share of each factor in the initial adaptation cost of this road segment
    tot_rel_share : list
        relative share of each factor in the total adaptation cost of this road segment
    bc_ratio : list
        benefit cost ratios for this road segment

    """
    # Identify terrain type of the road
    if x.terrain.lower().strip() == 'mountain' or x.asset_type == 'Bridge':
        ter_type = 'mountain'
    elif x.terrain.lower().strip() == 'flat':
        ter_type = 'flat'

    rehab_costs['rate_m'] = length_factor*rehab_costs.basic_cost
    # Identify asset type, which is the main driver of the costs
    if (x.asset_type == 'Expressway') | ((national == True) & (x.road_class == 1)):
        rehab_cost = rehab_costs.loc[('Expressway', ter_type), 'rate_m']
        rehab_corr = rehab_costs.loc[('Expressway', ter_type), 'design_width']
    elif (x.asset_type == 'National roads') | ((national == True) & (x.road_class == 2)):
        rehab_cost = rehab_costs.loc[('National  2x Carriageway', ter_type), 'rate_m']
        rehab_corr = rehab_costs.loc[('National  2x Carriageway', ter_type), 'design_width']
    elif (x.asset_type == 'National roads') | ((national == True) & (x.road_class == 3)):
        rehab_cost = rehab_costs.loc[('National  1x Carriageway', ter_type), 'rate_m']
        rehab_corr = rehab_costs.loc[('National  1x Carriageway', ter_type), 'design_width']
    elif (x.asset_type == 'Provincial roads') | ((national == True) & (x.road_class == 4)):
        rehab_cost = rehab_costs.loc[('Provincial', ter_type), 'rate_m']
        rehab_corr = rehab_costs.loc[('Provincial', ter_type), 'design_width']
    elif ((x.asset_type == 'Urban roads/Named roads') | (x.asset_type == 'Boulevard')) | ((national == True) & (x.road_class == 5)):
        rehab_cost = rehab_costs.loc[('District', ter_type), 'rate_m']
        rehab_corr = rehab_costs.loc[('District', ter_type), 'design_width']
    elif (x.asset_type == 'Other roads') | ((national == True) & (x.road_class == 6)):
        rehab_cost = rehab_costs.loc[('Commune', ter_type), 'rate_m']
        rehab_corr = rehab_costs.loc[('Commune', ter_type), 'design_width']
    elif x.asset_type == 'Bridge':
        rehab_cost = rehab_costs.rate_m.max()
        rehab_corr = rehab_costs.design_width.max()
    else:
        rehab_cost = rehab_costs.rate_m.min()
        rehab_corr = rehab_costs.design_width.min()

    rehab_cost = x.width*rehab_cost/rehab_corr

    return rehab_cost

Tanzania Cost Preprocessing

In [79]:
# Define paths
data_path = "/Volumes/IT/git_projects/traction/casestudies/tanzania/data/"
road_path = "/Volumes/IT/git_projects/traction/casestudies/tanzania/data/networks/road.gpkg"
rail_path = "/Volumes/IT/git_projects/traction/casestudies/tanzania/data/networks/rail.gpkg"

In [56]:
# Read rail costs
costs_path = os.path.join(data_path,"costs","road_and_rail_costs.xlsx")
rail_costs = pd.read_excel(costs_path, sheet_name = "rail_costs")

rail_edges = gpd.read_file(rail_path, layer = "edges")

rail_edges = rail_edges.to_crs(epsg=4326)
rail_edges = rail_edges.to_crs(epsg=32736)

rail_edges['length_m'] = rail_edges.apply(lambda x:x.geometry.length,axis=1)

### Add rail costs 

# Set cost_unit
rail_edges["cost_unit"] = "USD/m"
# Set cost_min
rail_edges["cost_min"] = rail_costs.cost_min[0] * 0.001
# Set cost_max
rail_edges["cost_max"] = rail_costs.cost_max[0] * 0.001

### Export to file
rail_edges.to_file(rail_path, layer='edges', driver='GPKG')

In [ ]:
# Rail costs

In [120]:
costs_path = os.path.join(data_path,"costs","road_and_rail_costs.xlsx")
rail_costs = pd.read_excel(costs_path, sheet_name = "rail_costs")

In [119]:
rail_edges = gpd.read_file(rail_path, layer = "edges")

In [136]:
rail_edges["cost_min"] = float(rail_costs.cost_min.values[0])
rail_edges["cost_max"] = float(rail_costs.cost_max.values[0])
rail_edges["cost_unit"] = str(rail_costs.cost_unit.values[0])

In [138]:
rail_edges.to_file(rail_path, layer='edges', driver='GPKG')

In [72]:
# Road costs

In [108]:
def get_road_rehab_costs(x, rehab_costs):
    if x.roadclass == "T":
        highway = "trunk"
    if x.roadclass == "R":
        highway = "primary"
    if x.roadclass  == "N":
        highway = "primary"
    
    condition = x.road_cond
    
    cost_min = rehab_costs.cost_min.loc[(rehab_costs.highway==highway)&(rehab_costs.road_cond==condition)].values
    cost_max = rehab_costs.cost_max.loc[(rehab_costs.highway==highway)&(rehab_costs.road_cond==condition)].values
    cost_unit = rehab_costs.cost_unit.loc[(rehab_costs.highway==highway)&(rehab_costs.road_cond==condition)].values
    
    return float(cost_min[0]) , float(cost_max[0]) , str(cost_unit[0])

In [109]:
costs_path = os.path.join(data_path,"costs","road_and_rail_costs.xlsx")
road_costs = pd.read_excel(costs_path, sheet_name = "road_costs")

In [110]:
road_edges = gpd.read_file(road_path, layer = "edges")
road_edges["rehab_costs"] = road_edges.progress_apply(lambda x:get_road_rehab_costs(x,road_costs),axis=1)
road_edges[["cost_min","cost_max","cost_unit"]] = road_edges["rehab_costs"].progress_apply(pd.Series)
road_edges.drop("rehab_costs",axis=1,inplace=True)

In [115]:
road_edges.to_file(road_path, layer='edges', driver='GPKG')